# Les 2 Toepassing Bag of Words
In dit notebook kijken we naar een toepassing van het bag of words model. Eerst wordt er gekeken of de zoektermen van de abstracts ook daadwerkelijk meer voorkomen tussen de 2 verschillende onderzoeksonderwerpen, hiermee wordt een scheiding van de zoektermen verwacht per type abstract.
Daarna wordt vanuit sklearn een naive_bayes classificatie model geimporteerd om te kijken of de verschillende types BoW-encodings impact hebben op de prestatie van een machine learning model.

In [8]:
files = ["../resources/cancer_abstract.txt","../resources/cancer_2_abstract.txt","../resources/fibomalgia_2_abstract.txt","../resources/fibromalgia_abstract.txt"]

Voor het maken van de bag of words wordt eerst de tekst samengevoegd, getokeniseerd en daarna uit elkaar gehaald. Hiervoor gebruiken we het argument 2000 max tokens met een minimale frequentie van 2, zodat we een uitgebreide corpus krijgen.

In [10]:
from nlp import file_merger,group_encoder,multi_hot_encoding,frequency_checker,tf_idf_calc
max_tokens = 2000
min_freq = 2
merged_words, len_of_files = file_merger(files)
uncoupled_token_list_of_lists, token_dict = group_encoder(max_tokens,min_freq,merged_words,len_of_files)
print(token_dict)

{1: 'c', 2: 'a', 3: 'n', 4: 'e', 5: 'r', 6: 'l', 7: 's', 8: 'u', 9: 'm', 10: 't', 11: 'y', 12: 'g', 13: 'i', 14: 'o', 15: 'h', 16: 'f', 17: 'b', 18: 'w', 19: 'd', 20: 'v', 21: 'p', 22: ',', 23: '.', 24: 'j', 25: 'k', 26: 'x', 27: 'z', 28: '-', 29: 'q', 30: '(', 31: ')', 32: '1', 33: '8', 34: '9', 35: '0', 36: '%', 37: '—', 38: ';'}


## De zoektermen in tokens  
cancer mutation en fibromalgia  
  
cancer 109  
mutation token 223  
fibromalgia token 395  

Hieronder de generatie van de BoW encodings in een dataframe format.

In [11]:
multi_df = multi_hot_encoding(uncoupled_token_list_of_lists,token_dict,files)
multi_df.head()

,../resources/cancer_abstract.txt,../resources/cancer_2_abstract.txt,../resources/fibomalgia_2_abstract.txt,../resources/fibromalgia_abstract.txt
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
5,1,1,1,1


In [12]:
freq_df = frequency_checker(uncoupled_token_list_of_lists,token_dict,files," ")
freq_df.head()

,../resources/cancer_abstract.txt,../resources/cancer_2_abstract.txt,../resources/fibomalgia_2_abstract.txt,../resources/fibromalgia_abstract.txt
1,30,27,52,43
2,51,53,119,87
3,57,57,95,78
4,68,48,130,87
5,44,25,75,56


In [13]:
tf_df = tf_idf_calc(uncoupled_token_list_of_lists,token_dict,files)
tf_df.head()

,../resources/cancer_abstract.txt,../resources/cancer_2_abstract.txt,../resources/fibomalgia_2_abstract.txt,../resources/fibromalgia_abstract.txt
1,0.149571,0.144148,0.120695,0.134520
2,0.197031,0.219260,0.214028,0.210900
3,0.227131,0.243218,0.176232,0.195024
4,0.255044,0.192782,0.226991,0.204747
5,0.200355,0.121901,0.158989,0.160003


## Zoektermen vergelijken met TF-IDF  
Voor het vergelijken van de zoektermen in elk bestand gaan we de TF-IDF waardes vergelijken in het tf_df dataframe  

In [14]:
# token key -1 voor indexing
cancer = 109-1
mutation = 223-1
fibromalgia = 395-1
tf_df.iloc[[cancer,mutation,fibromalgia]]


IndexError: positional indexers are out-of-bounds

Zoals te zien aan het dataframe is er een duidelijk verschil aan TF-IDF waardes tussen de cancer en fibromalgia bestanden

## Machine learning op Bag of Words data
Om te kijken of het type BoW encoding verschil uitmaakt voor het trainen van de data, wordt er op welk van de datasets een GaussianNB model getraind waar alle data van kanker True is een van fibromalgia False.

In [16]:
import numpy as np

kanker_file_een = tf_df[[files[0]]].to_numpy()
kanker_file_twee = tf_df[[files[1]]].to_numpy()
# data,labels
kanker_files = np.concatenate((kanker_file_een, kanker_file_twee))
kanker_labels = np.ones(len(kanker_files)).astype(bool)

fibro_file_een = tf_df[[files[2]]].to_numpy()
fibro_file_twee = tf_df[[files[3]]].to_numpy()
fibro_files = np.concatenate((fibro_file_een,fibro_file_twee))
fibro_labels = np.zeros(len(fibro_files)).astype(bool)

labels = np.concatenate((kanker_labels,fibro_labels))
data = np.concatenate((kanker_files,fibro_files))

In [17]:
from sklearn import naive_bayes
from sklearn.model_selection import cross_validate
print("GaussianNB op tf-idf encoding")
model = naive_bayes.GaussianNB()
score = cross_validate(model,data,labels, return_train_score=True)
for key, val in score.items():
    print(key , val.mean())


GaussianNB op tf-idf encoding
fit_time 0.0011019706726074219
score_time 0.0006243705749511719
test_score 0.5
train_score 0.5247391952309984


freq diff

In [18]:
kanker_file_een = freq_df[[files[0]]].to_numpy()
kanker_file_twee = freq_df[[files[1]]].to_numpy()
kanker_files = np.concatenate((kanker_file_een, kanker_file_twee))
kanker_labels = np.ones(len(kanker_files)).astype(bool)

fibro_file_een = freq_df[[files[2]]].to_numpy()
fibro_file_twee = freq_df[[files[3]]].to_numpy()
fibro_files = np.concatenate((fibro_file_een,fibro_file_twee))
fibro_labels = np.zeros(len(fibro_files)).astype(bool)

labels = np.concatenate((kanker_labels,fibro_labels))
data = np.concatenate((kanker_files,fibro_files))

In [19]:
model = naive_bayes.GaussianNB()
score = cross_validate(model,data,labels, return_train_score=True)
print("GaussianNB op freqwentie encoding")
for key, val in score.items():
    print(key , val.mean())

GaussianNB op freqwentie encoding
fit_time 0.0006382942199707031
score_time 0.0005040168762207031
test_score 0.5653763440860214
train_score 0.5805988348462268


multi def

In [20]:
kanker_file_een = multi_df[[files[0]]].to_numpy()
kanker_file_twee = multi_df[[files[1]]].to_numpy()
kanker_files = np.concatenate((kanker_file_een, kanker_file_twee))
kanker_labels = np.ones(len(kanker_files)).astype(bool)

fibro_file_een = multi_df[[files[2]]].to_numpy()
fibro_file_twee = multi_df[[files[3]]].to_numpy()
fibro_files = np.concatenate((fibro_file_een,fibro_file_twee))
fibro_labels = np.zeros(len(fibro_files)).astype(bool)

labels = np.concatenate((kanker_labels,fibro_labels))
data = np.concatenate((kanker_files,fibro_files))

In [21]:
model = naive_bayes.GaussianNB()
print("GaussianNB op multi-hot encoding")
score = cross_validate(model,data,labels, return_train_score=True)
for key, val in score.items():
    print(key , val.mean())

GaussianNB op multi-hot encoding
fit_time 0.0005207538604736328
score_time 0.00047001838684082033
test_score 0.5531182795698925
train_score 0.5526622408887685


## Vergelijking
Zoals te zijn aan de bovenstaande resultaten zit er geen verschil in test en train scores die de prestatie van het model weergeven. Wel is hier op aan te merken dat de deze scores onder de 0.7 zijn en dus niet voldoen als een praktisch implementeerbaar model. Waar de encodings wel een verschil maken is in de trainingstijd, hierin waren multi-hot en frequentie encoding een stuk sneller dan de tf-idf data. Dit zal waarschijnlijk komen omdat de getallen bij multi-hot en frequentie encoding simpler zijn(int) dan de tf-idf getallen(float).